In [12]:
###코드 통합

################################################################

# 1. 라이브러리 선언
# 2. 전역변수 선선
# 3. 공용 메소드 설정
# 4. 클래스 선언부
# 4-1. 중고차 크롤링 선언
# 4-1-1. kcar 크롤링 선언
# 4-2. 신차 크롤링 선언
# 5. main 작동부

#################################################################

# 1. 라이브러리 선언

from selenium import webdriver
import pandas as pd
import time
import requests, bs4
import csv

# 2. 전역변수 정리

# 3. 공용 메소드 선언
# 3-1. selenium 시동

def startSelenimum():
    
    driverpath = "./chromedriver.exe"
    options = webdriver.ChromeOptions()

    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument('disable-gpu')

    # 웹드라이버 정의(위 headless 사용시 필수)

    driver = webdriver.Chrome(driverpath, options=options)
    
    return driver

def trigerSelenium(driver,url):

    driver.get(url)
    print("페이지 생성")

# 3-2. selenium parser
def getHtmlWithSelenium(driver_tool, targetUrl):

    driver_tool.get(targetUrl)

    time.sleep(0.3)

    html = driver_tool.page_source

    bs = bs4.BeautifulSoup(html,"html.parser")

    return bs
# 3-3. request parser
def getHtmlWithRequests(targetUrl, sleepSeconds):

    response = requests.get(url = targetUrl)

    time.sleep(sleepSeconds)

    response.encoding = "utf-8"

    html = response.text

    bs = bs4.BeautifulSoup(html, "html.parser")

    return bs


class getURL:
    """"Super Class"""
    def __init__(self, baseurl):
        self.baseurl = baseurl




##본 코드는 kcar 한정코드
# 4-1-1. kcarurl 코드 부분
class kCar_getURL(getURL):
    """sub Class"""
    def __init__(self, baseurl):


        self.baseurl = baseurl

        

        
    def option(driver):

        cntMainXpath = '//*[@id="filterSearch"]/section[2]/div[2]/div[3]/div[2]/b'
        cntSubXpath = '//*[@id="filterSearch"]/section[2]/div[2]/div[3]/div[3]/div/ul/li[4]'


        cntMain = driver.find_element_by_xpath(cntMainXpath)
        cntSub = driver.find_element_by_xpath(cntSubXpath)

        cntMain.click()
        time.sleep(delaychecker)
        cntSub.click()

    def collectUrl(delaychecker, driver):

        page = 0
        escape = 1
        print("페이지 조건 등록")
        time.sleep(delaychecker)

        html = driver.page_source
        bs = bs4.BeautifulSoup(html, 'html.parser')
        print("페이지 긁기 완료")
        startTime = time.time()

        urlList = []
        print("반복문 전까지 완료")
        while (escape == 1):
            for i in range(0, 10):
                try:

                    page += 1
                    pageXpath = '//*[@id="filterSearch"]/section[2]/div[5]/div/button[' + str(i + 1) + ']'
                    driver.find_element_by_xpath(pageXpath).click()
                    time.sleep(delaychecker)
                    html = driver.page_source
                    bs = bs4.BeautifulSoup(html, 'html.parser')
                    buttonTags = bs.find(name = "ol").findAll(name = "button", attrs = {"class" : "btn_compare"})


                    for eachButtonTag in buttonTags:

                        addUrl = eachButtonTag['data-car-cd']
                        eachUrl = baseUrl + addUrl
                        urlList.append(eachUrl)

                except:

                    escape = 0

            if (page % 10 == 0):

                try:

                    nextIndexXpath = '//*[@id="filterSearch"]/section[2]/div[5]/button[2]'
                    driver.find_element_by_xpath(nextIndexXpath).click()
                    time.sleep(delaychecker)

                except:

                    escape = 0

        endTime = time.time()
        doneTime = endTime - startTime
        print("총 소요시간" + doneTime)
        print("총 URL은 " + len(urlList))

        linkDataFrame = pd.DataFrame(urlList, columns = ['url'])
        linkDataFrame.to_csv("./UrlList.csv", encoding='ms949', index=False)
        urlList = urlDataFrame['url'].tolist()

        return urlList

# 5. main part
try:
    delaychecker = 0.3
    kCarUrl = "https://www.kcar.com/car/search/car_search_list.do"
    seldriver = startSelenimum()
    print("드라이버 생성")
    trigerSelenium(seldriver, kCarUrl)
    print("셀레니움 가동")
    kCarpage = kCar_getURL(kCarUrl)
    print("페이지 정상작동")
    
   
    #kcar객체선언

#     #kcar의 페이지 옵션(60페이지 검색 클릭)
#     kCarpage.option(seldriver)

except:
    print("페이지 불러오기 오류")



#상세정보url 리스트 정리

kCarUrlList = kCarpage.collectUrl(delaychecker, seldriver)





## 원본 main 코드 부분

containingList =[]
startTime = time.time()
retryTargetUrlList = []
totalList=[]


for targetUrl in kCarUrlList:
    try:
        containingList = []
        request_bs = getHtmlWithRequests(targetUrl, delaychecker)
        kCar_getInfo(request_bs, containingList)
        totalList.append(containingList)
        
    except:
        retryTargetUrlList.append(targetUrl)
        print("request 페이지 정보 parsing 실패")
        print(targetUrl)
endTime = time.time()
doneTime = endTime - startTime
print("request parsing time"+ doneTime)

result = pd.DataFrame(totalList)

result.columns = ['제품명', '사고유무', '중고차 가격', '차량번호', '배기량', '연식', '색상', '주행거리', '변속기', '연료', '차종', '압류/저당', '구동방식', '인승', '제시번호']

result.to_csv('./UsedCarData.csv', encoding = 'ms949', index = False)

retryTargetUrlDataFrame = pd.DataFrame(retryTargetUrlList)
retryTargetUrlDataFrame.columns = ["Url"]
retryTargetUrlDataFrame.to_csv('./retryTargetUrl.csv', encoding = 'ms949', index = False)


retryTargetUrlDataFrame = pd.read_csv("./retryTargetUrl.csv", encoding = 'ms949')
retryTargetUrlList = retryTargetUrlDataFrame['Url'].tolist()

errorURL = []
reTotalList=[]
startTime = time.time()
for eachRetryUrl in retryTargetUrlList:#여기를 retryTargetUrl로 변경해야함

    try:
        containingList = []
        selenium_bs = getHtmlWithSelenium(driver_tool, eachRetryUrl)
        containingList = kCar_getInfo(selenium_bs, containingList)
        reTotalList.append(eachRetryUrl)

    except:
        errorURL.append(targetUrl)
        print("selenium 페이지 정보 parsing 실패")
        print(targetUrl)

endTime = time.time()
doneTime = endTime - startTime
print("selenium parsing time"+ doneTime)



reResult = pd.DataFrame(reTotalList)#없는 변수, 해당부분을 containinglist로 변경필

reResult.columns = ['제품명', '사고유무', '중고차 가격', '차량번호', '배기량', '연식', '색상', '주행거리', '변속기', '연료', '차종', '압류/저당', '구동방식', '인승', '제시번호']

finaldata = pd.concat([result, reResult])

finaldata.to_csv("./filename.csv", mode='w')


#현제 셀레니움과 리퀘스트 관련하여 리스트로 붙여넣고 있음, for문활용하여 dataframe 형태로 변경필요